<a href="https://colab.research.google.com/github/Anna671/NLP/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importing libraries 
import pandas as pd
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, GRU, SimpleRNN
from keras.datasets import imdb

In [ ]:
from keras.callbacks import History

In [ ]:
history = History()

In [ ]:
df = pd.read_csv('../input/nlp-getting-started/train.csv')

In [ ]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
#Deleting columns
df.drop(['keyword', 'location'], axis=1, inplace=True)

In [ ]:
df

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


In [ ]:
def max_lenth(max_l):
    max_l = 150
    for i in x_train:
        if len(i) > max_l: max_l = len(i)
    return max_l
max_l = max_lenth(df['text'])
max_l

157

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
                                    df['text'], df['target'], test_size=0.33, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((5100,), (2513,), (5100,), (2513,))

In [ ]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [ ]:
# Choose the top 600 words from the vocabulary
top_k = 600
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(x_train)

In [ ]:
# Create the tokenized vectors
x_train_seqs = tokenizer.texts_to_sequences(x_train)
X_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_seqs, maxlen=max_l)

In [ ]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
x_test_seqs = tokenizer.texts_to_sequences(x_test)
X_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_seqs, maxlen=max_l)

In [ ]:
X_train.shape,y_train.shape

((5100, 157), (5100,))

In [ ]:
import tensorflow as tf
import keras.backend as K
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, GRU, SimpleRNN
from keras.datasets import imdb

In [ ]:
model = Sequential()

model.add(Embedding(top_k, 600))
model.add(LSTM(25, dropout = 0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy', 'AUC'])

In [ ]:
#Training
%%time
history = model.fit(X_train, y_train, 
                    batch_size = 5,
                    epochs = 20,
                    validation_data = (X_test, y_test),
                    verbose = 1, callbacks=[history]   )

Epoch 1/20
1020/1020 [==============================] - 79s 76ms/step - loss: 0.5857 - accuracy: 0.6798 - auc: 0.7317 - val_loss: 0.4608 - val_accuracy: 0.7971 - val_auc: 0.8476
Epoch 2/20
1020/1020 [==============================] - 76s 75ms/step - loss: 0.3787 - accuracy: 0.8464 - auc: 0.9011 - val_loss: 0.4659 - val_accuracy: 0.7895 - val_auc: 0.8504
Epoch 3/20
1020/1020 [==============================] - 76s 74ms/step - loss: 0.3358 - accuracy: 0.8579 - auc: 0.9226 - val_loss: 0.4900 - val_accuracy: 0.7855 - val_auc: 0.8363
Epoch 4/20
1020/1020 [==============================] - 77s 76ms/step - loss: 0.2814 - accuracy: 0.8891 - auc: 0.9443 - val_loss: 0.5867 - val_accuracy: 0.7776 - val_auc: 0.8353
Epoch 5/20
1020/1020 [==============================] - 79s 77ms/step - loss: 0.2494 - accuracy: 0.8961 - auc: 0.9567 - val_loss: 0.6514 - val_accuracy: 0.7728 - val_auc: 0.8291
Epoch 6/20
1020/1020 [==============================] - 75s 74ms/step - loss: 0.2083 - accuracy: 0.9115 - auc:

In [ ]:
model.evaluate(X_test,y_test,batch_size=5)

503/503 [==============================] - 6s 11ms/step - loss: 1.1654 - accuracy: 0.7477 - auc: 0.7982


[1.1653943061828613, 0.7477118968963623, 0.798201322555542]

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred_val = np.round(model.predict(X_valid)).astype(int)
y_pred_val

array([[1],
       [1],
       [0],
       ...,
       [1],
       [1],
       [0]])

## Validation prediction 

In [ ]:
df_test = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
df_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [ ]:
x_valid = df_test['text']

In [ ]:
x_valid_seqs = tokenizer.texts_to_sequences(x_valid)
X_valid = tf.keras.preprocessing.sequence.pad_sequences(x_valid_seqs, maxlen=max_l)

In [ ]:
y_pred_val = np.round(model.predict(X_valid)).astype(int)
y_pred_val

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [ ]:
df_test['target']=y_pred_val
df_test[['id','target']].to_csv('sample_submission2.csv',index=False)

keras_preprocessing.text.Tokenizer